In [2]:
#cell 1
# =============================
# 数据准备:原子化拆解
# =============================
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import seaborn as sns
from dotenv import load_dotenv
import os

# 加载本地.env文件
load_dotenv()

# 从环境变量读取连接信息
USER = os.getenv("DB_USER")
PASSWORD = os.getenv("DB_PASS")
HOST = os.getenv("DB_HOST","localhost")
PORT = os.getenv("DB_PORT","5432")
DB = os.getenv("DB_NAME")

# 检查读取是否成功(调试用)
if not PASSWORD:
    print("警告:未找到数据库密码,请检查.env文件")
else:
    print("配置加载成功(具体信息已隐藏)")

engine = create_engine(f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB}")

# 拉取全量原子数据
df_items = pd.read_sql("SELECT * FROM analysis.analysis_items_atomic", engine)
print(f"✅ 原子化数据准备完成。行数: {len(df_items)} (比订单数多，因为包含了 Mixed Basket)")
df_items.head()

配置加载成功(具体信息已隐藏)
✅ 原子化数据准备完成。行数: 110189 (比订单数多，因为包含了 Mixed Basket)


,order_id,order_item_id,product_id,seller_id,price,category,review_score,delay_days,is_late
0,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,pet_shop,4,-2.0,0
1,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,furniture_decor,5,-13.0,0
2,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,garden_tools,5,-15.0,0
3,00048cc3ae777c65dbb7d2a0634bc1ea,1,ef92defde845ab8450f9d70c526ef70f,6426d21aca402a131fc0a5d0960a3c90,21.90,housewares,4,-14.0,0
4,00054e8431b9d7675808bcb819fb4a32,1,8d4f2bb7e93e6710a28f34fa83ee7d28,7040e82f899a04d1b434b795a43b4617,19.90,telephony,4,-16.0,0


In [3]:
#cell 2
# 验证"钩子商品(复购率高的商品)" (基于 Item 粒度)

# =============================
# 1. 找出复购用户 ID
# =============================
df_rfm = pd.read_sql("SELECT * FROM analysis.analysis_user_rfm", engine)
df_obt = pd.read_sql("SELECT * FROM analysis.analysis_orders_obt", engine)
repurchase_user_ids = df_rfm[df_rfm["frequency"] > 1]["user_id"]

# ==============================
# 2. 关联 Item 表，找到这些复购用户的"首单"里包含的所有品类
# ==============================
first_orders = df_obt.sort_values("purchase_ts").groupby("user_id")["order_id"].first()
repurchase_first_order_ids = first_orders[first_orders.index.isin(repurchase_user_ids)]

# 筛选出这些首单包含的所有 Item
hook_items = df_items[df_items["order_id"].isin(repurchase_first_order_ids)]

# 统计品类出现次数 (Item Count)
hook_counts = hook_items["category"].value_counts()

print("\n🔍 [可行性验证 - 原子粒度]")
print(f"复购用户首单包含的商品总数: {len(hook_items)}")
print(hook_counts.head(10))



🔍 [可行性验证 - 原子粒度]
复购用户首单包含的商品总数: 3416
category
bed_bath_table              498
furniture_decor             393
sports_leisure              303
health_beauty               247
computers_accessories       235
housewares                  188
garden_tools                133
watches_gifts               130
telephony                   114
fashion_bags_accessories    113
Name: count, dtype: int64
